# Reading and tokenizing data

In [13]:
import nltk
from typing import Iterator, List
from nltk import RegexpTokenizer, sent_tokenize
from tqdm import tqdm
from pathlib import Path

nltk.download('punkt')

import re
_patterns = [r"<br \/>", r",", r"\(", r"\)", r"\„", r"\“"]

_replacements = [" ", " , ", " ( ", " ) ", " \" ", " \" "]

_patterns_dict = list((re.compile(p), r) for p, r in zip(_patterns, _replacements))


def generate_tokenized_sentences(path: Path, paragraphs=1_000_000) -> Iterator[str]:
    """
    Tokenize paragraphs into sentences, and sentences into words.
    :param paragraph: text of paragraph
    """
    word_tokenizer = RegexpTokenizer(r'\b\d+[/\.]\d+[/\.]\d+|ძვ\.წ\.|ახ\.წ\.|ე\.წ\.|ა\.შ\.|ე\.ი\.|[-\w]+|[-!\".#$%&\'()*+,/:;<=>?@\[\\\]^_`{|}~„“+]+')

    # for sentence in sent_tokenize(paragraph):
    with open(path, 'r') as file:
        for i, paragraph in enumerate(file):
            if i >= paragraphs:
                break
                
            paragraph = paragraph.lower()
            for pattern_re, replaced_str in _patterns_dict:
                paragraph = pattern_re.sub(replaced_str, paragraph)

            tokenized_sentence = word_tokenizer.tokenize(paragraph)
            if tokenized_sentence:
                start = 0
                for i, tkn in enumerate(tokenized_sentence):
                    if tkn in ('.', '!', '?'):
                        yield tokenized_sentence[start: i+1]
                        start = i+1

                if start < len(tokenized_sentence):
                    yield tokenized_sentence[start:]

[nltk_data] Downloading package punkt to /home/khokho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
from pathlib import Path

temp_data_path = Path('data/ka_nse_mil.txt')

tokenized_sentences = [sent for sent in generate_tokenized_sentences(temp_data_path)]
tokenized_sentences[0]

['ძეგლთა',
 'დაცვის',
 'ეროვნულ',
 'სააგენტოში',
 'აცხადებენ',
 ',',
 'რომ',
 'მათ',
 'მიერ',
 '"',
 'სამხრეთის',
 'კარიბჭესთვის',
 '"',
 'მოწოდებული',
 'ინფორმაცია',
 ',',
 'იმის',
 'შესახებ',
 'რომ',
 'ახალციხის',
 'არქივის',
 'ეზოში',
 'აღმოჩენილი',
 'შენობა',
 'საკულტო',
 'ნაგებობა',
 'იყო',
 'არასწორია',
 'და',
 'საბოლოოდ',
 'დადგინდა',
 ',',
 'რომ',
 'აქ',
 'ტრადიციული',
 'მესხური',
 'საცხოვრებელი',
 'იყო',
 '.']

# Training the n-gram language model

In [15]:
# import nltk
# from nltk.util import everygrams
# from nltk.lm.preprocessing import pad_both_ends

In [16]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline

# adds padding to sentences, on the left and on the right
# train contains ngrams (if n=3, train contains 1-grams, 2-grams and 3-grams)
# vocab contains padded and flattened sentences
temp_data_path = Path('data/ka.txt')
n = 3
train, vocab = padded_everygram_pipeline(n, tokenized_sentences)

In [17]:
from nltk.lm import MLE, Laplace
# model = MLE(n)
# TODO: compare different lm models (laplace seems better than mle)
model = Laplace(n)

In [18]:
model.fit(train, vocab)
print(model.vocab)
print(model.counts)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 470950 items>
<NgramCounter with 3 ngram orders and 25665363 ngrams>


In [37]:
tst_sent = next(generate_tokenized_sentences(temp_data_path))
print(model.vocab.lookup(tst_sent))
print(model.vocab.lookup(tst_sent + ['salfknoonqwf'])) # check unknown tokens

('ეროვნული', 'სააგენტო', ':', '"', 'ახალციხეში', 'აღმოჩენილი', 'შენობა', 'არა', 'საკულტო', 'ნაგებობა', ',', 'არამედ', 'საყოფაცხოვრებო', 'დანიშნულების', 'იყო', '"')
('ეროვნული', 'სააგენტო', ':', '"', 'ახალციხეში', 'აღმოჩენილი', 'შენობა', 'არა', 'საკულტო', 'ნაგებობა', ',', 'არამედ', 'საყოფაცხოვრებო', 'დანიშნულების', 'იყო', '"', '<UNK>')


In [20]:
# TODO: add more visualizations

In [21]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sentence(model, num_words=20, text_seed=None, random_seed=None):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max num of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []

    for token in model.generate(num_words, text_seed=text_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)

    return detokenize(text_seed + content) if text_seed else detokenize(content)

In [22]:
generate_sentence(model, num_words=20, text_seed=['ეს', 'არის'], random_seed=78)

'ეს არის ფილმი, უკლებრივ არის შედევრი!'

In [23]:
# random sentence
generate_sentence(model, num_words=20)

'ბ.'

# Saving the model

In [24]:
from pathlib import Path

ngram_dir = Path("models/n-gram")
ngram_dir.mkdir(parents=True, exist_ok=True)

In [25]:
import pickle 

with open(ngram_dir / 'ngram.pkl', 'wb') as fout:
    pickle.dump(model, fout)

In [26]:
import pickle 

with open(ngram_dir / 'ngram.pkl', 'rb') as fin:
    model_loaded = pickle.load(fin)

In [27]:
# Pickled model generates the same sentence when using the same parameters.
generate_sentence(model_loaded, num_words=20, text_seed=['ეს', 'არის'], random_seed=78)

'ეს არის ფილმი, უკლებრივ არის შედევრი!'

In [28]:
generate_sentence(model_loaded, num_words=100, text_seed=['ჩემი', 'ხატია', 'სამშობლო'], random_seed=78)

'ჩემი ხატია სამშობლო პირველად გამოჩნდა წარწერა " დავითი, - ამის შესახებ საგამოძიებო კომისიის წევრები აცხადებენ, რომ გურამ ვერულიძე, ტენდერში გამარჯვებული მეწარმეებისაგან რეგულარულად იღებდა ქრთამს.'

In [29]:
# Pickled model generates the same sentence when using the same parameters.
generate_sentence(model_loaded, num_words=20, text_seed=['ქართველი', 'ერი', 'ამას', 'არ', 'აიტანს'], random_seed=78)

'ქართველი ერი ამას არ აიტანს მას სართულებზე.'

In [30]:
# Pickled model generates the same sentence when using the same parameters.
generate_sentence(model_loaded, num_words=20, text_seed=['ავადმყოფი', 'ერია', 'ქართველები'])

'ავადმყოფი ერია ქართველები მიჩვეული ვართ ყველაფრის ისე გაგებას, ბრმები დანახვას, პარალიზებული ადამიანები.'

In [31]:
generate_sentence(model_loaded, num_words=100, text_seed=['გიორგი', 'სახლში'], random_seed=78)

'გიორგი სახლში ტაქსით წამოვედი.'

In [32]:
generate_sentence(model_loaded, num_words=100, text_seed=['რა', 'გინდა'], random_seed=78)

'რა გინდა მოსკოვში?'

In [35]:
for i in range(5):
    print(generate_sentence(model_loaded, num_words=100, text_seed=['გიორგი', 'სახლში']))

გიორგი სახლში გარდაცვლილი იპოვეს.
გიორგი სახლში უსაფრთხოებას 7 მარტის ღამიდან იცავენ.
გიორგი სახლში " დაიხურება.
გიორგი სახლში ფეხით მაცილებს.
გიორგი სახლში შეხვდებით, გადამეტებული აქტიურობა, თუკი თქვენს ხელისგულზე ორზე მეტი.


In [38]:
test_data_path = Path('data/ka_nse_test.txt')
test_tokenized_sentences = [sent for sent in generate_tokenized_sentences(test_data_path)]
test_tokenized_sentences[0]

['1912',
 'წლის',
 'მარტში',
 'ნეაპოლის',
 'ნავსადგურში',
 'დიდი',
 'საოკეანო',
 'გემის',
 'გადმოტვირთვისას',
 'მოხდა',
 'ერთი',
 'უცნაური',
 'უბედური',
 'შემთხვევა',
 ',',
 'რომლის',
 'თაობაზე',
 'გაზეთებმა',
 'დაწვრილებით',
 ',',
 'მაგრამ',
 'ძალზე',
 'შელამაზებული',
 'ცნობები',
 'გამოაქვეყნეს',
 '.']

In [39]:
from tqdm import tqdm

test_data, _ = padded_everygram_pipeline(n, test_tokenized_sentences)
perplexities = []

for test in tqdm(test_data):
    perplexities.append(model.perplexity(test))

128050it [00:39, 3251.28it/s]


In [42]:
import numpy as np

print(f'Average perplexity: {np.mean(perplexities)}\n')

ids_sorted = np.argsort(perplexities)
print(f'Non-trivial sentences with lowest perplexity:')
p = 0
for i in range(99999999):
    ii = ids_sorted[i]
    if len(test_tokenized_sentences[ii]) > 10:
        p += 1
        print(f'{" ".join(test_tokenized_sentences[ii])}\n\tScore: {perplexities[ii]}')
    if p >= 5:
        break

print(f'\nNon-trivial sentences with highest perplexity:')
p = 0
for i in range(99999999):
    ii = ids_sorted[-i]
    if len(test_tokenized_sentences[ii]) > 10:
        p += 1
        print(f'{" ".join(test_tokenized_sentences[ii])}\n\tScore: {perplexities[ii]}')
    if p >= 5:
        break

Average perplexity: 11146.680899092791

Non-trivial sentences with lowest perplexity:
- არა , არა , არა , არა , არა !
	Score: 441.9904338589656
- რა თქმა უნდა , რა თქმა უნდა , თქვა დედამ .
	Score: 533.3392371147456
- რა თქმა უნდა , ვიცი , რომ მდიდარი ხარ .
	Score: 594.9932345812091
- რა თქმა უნდა , არა , - მივუგე მე .
	Score: 603.2878588405559
- არა , სერ , რა თქმა უნდა , არა !
	Score: 619.0374309955182

Non-trivial sentences with highest perplexity:
aber statt dessen , wie nutzlos müht er sich ab ; immer noch zwängt er sich durch die gemächer des innersten palastes ; niemals wird er sie überwinden ; und gelänge ihm dies , nichts wäre gewonnen ; die treppen hinab müßte er sich kämpfen ; und gelänge ihm dies , nichts wäre gewonnen ; die höfe wären zu durchmessen ; und nach den höfen der zweite umschließende palast ; und wieder treppen und höfe ; und wieder ein palast ; und so weiter durch jahrtausende ; und stürzte er endlich aus dem äußersten tor - aber niemals , niemals kann es gesche